In [13]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

import datetime

In [14]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [15]:
actions = np.array(["bend","jump","walk"])

In [16]:
X = np.load("./X1.npy",allow_pickle = True)
y = np.load("./y1.npy", allow_pickle = True)

#Chia dữ liệu
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [17]:
X.shape

(9, 30, 132)

In [18]:
X_train.shape

(7, 30, 132)

In [19]:
y_train.shape

(7, 3)

In [20]:
actions.shape[0]

3

In [21]:
import torch.optim as optim
import torch.nn as nn
from LSTM import LSTM_model
from torch.utils.data import Dataset, DataLoader
import torch

In [22]:
class MyDataset(Dataset):
    def __init__(self, data, labels = None):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        x = self.data[index]
        if self.labels is None:
            y = None
        else:
            y = self.labels[index]
        return x, y

    def __len__(self):
        return len(self.data)

In [23]:
# 创建数据集对象
dataset = MyDataset(X_train, y_train)
test = MyDataset(X_test, y_test)
# 创建数据加载器对象
train_loader = DataLoader(dataset, batch_size=4, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=4, shuffle=False, drop_last=True)

In [24]:
# 定义模型和优化器
model = LSTM_model(actions)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# 训练模型
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:  # 迭代训练数据
        optimizer.zero_grad()  # 梯度清零
        inputs = inputs.to(torch.float32)
        labels = labels.to(torch.float32)
        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)  # 计算平均损失
    print('Epoch: {}, Training Loss: {:.4f}'.format(epoch+1, epoch_loss))

torch.save(model, 'model.pth')

Epoch: 1, Training Loss: 0.3779
Epoch: 2, Training Loss: 0.3184
Epoch: 3, Training Loss: 0.2420
Epoch: 4, Training Loss: 0.1577
Epoch: 5, Training Loss: 0.1392
Epoch: 6, Training Loss: 0.1146
Epoch: 7, Training Loss: 0.0788
Epoch: 8, Training Loss: 0.0634
Epoch: 9, Training Loss: 0.0445
Epoch: 10, Training Loss: 0.0091
Epoch: 11, Training Loss: 0.0040
Epoch: 12, Training Loss: 0.0005
Epoch: 13, Training Loss: 0.0002
Epoch: 14, Training Loss: 0.0000
Epoch: 15, Training Loss: 0.0001
Epoch: 16, Training Loss: 0.0000
Epoch: 17, Training Loss: 0.0001
Epoch: 18, Training Loss: 0.0001
Epoch: 19, Training Loss: 0.0001
Epoch: 20, Training Loss: 0.0007
Epoch: 21, Training Loss: 0.0002
Epoch: 22, Training Loss: 0.0000
Epoch: 23, Training Loss: 0.0000
Epoch: 24, Training Loss: 0.0000
Epoch: 25, Training Loss: 0.0000
Epoch: 26, Training Loss: 0.0000
Epoch: 27, Training Loss: 0.0000
Epoch: 28, Training Loss: 0.0000
Epoch: 29, Training Loss: 0.0000
Epoch: 30, Training Loss: 0.0000
Epoch: 31, Training

In [26]:
for inputs, labels in test_loader:
    inputs = inputs.to(torch.float32)
    print(inputs.shape)
    res = model(inputs)
    for r in res:
        print("predict: {}".format(actions[np.argmax(r.detach().numpy())]))
    for label in labels:
        print("ground truth: {}".format(actions[np.argmax(label.numpy())]))